In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [ ]:
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender

evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective_function_pure_svd_item(trial):
    params = {
        'topK': trial.suggest_int('topK', 1, 1000, log=True),#1000
        'num_factors': trial.suggest_int('num_factors', 1, 1000, log=True) #1000
    }

    recommender_instance = PureSVDItemRecommender(URM_train)
    recommender_instance.fit(**params)
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[10, "MAP"]

In [ ]:
from optuna.samplers import RandomSampler
import optuna

database_path='hypertuning_results/tuning_pureSVD.db'


study=optuna.load_study(study_name='hyperparameters_pureSVD',
    storage=f'sqlite:///{database_path}'

)
study.sampler=RandomSampler(seed=SEED)
study.optimize(objective_function_pure_svd_item, n_trials=600, n_jobs=6)